In [1]:
import numpy as np, random, operator, pandas as pd, math

class Direction:
    def __init__(self, input_size, hidden_size, output_size):
        self.w1 = np.random.uniform(-1., 1., (input_size,hidden_size))
        self.w2 = np.random.uniform(-1., 1., (hidden_size,hidden_size))
        self.w3 = np.random.uniform(-1., 1., (hidden_size,output_size))
        
    def __repr__(self):
        return str(self.w1) + ',' + str(self.w2) + ',' + str(self.w3)
    
    #def __getitem__(self, key):
        #return getattr(self, key)
        #return self[key]
        
    def getWeights(self):
        return self.w1, self.w2, self.w3
    
    
class Fitness:
    def __init__(self, population):
        self.population = population
        self.bias = 1
        
        self.listSquares = []
        # 0 weights
        # 1 rect
        # 2 alive
        # 3 color
        # 4 distance
        
        self.fitnessResults = {}
        self.hasSquaresAlive = True
        self.bestDistance = 1000000
        
        
    def sigmoid(self, x, derivative=False):
        return x*(1.0-x) if derivative else 1.0/(1.0+np.exp(-x))
    
    # calcular a ação a ser executada seguindo a maior probabilidade
    def feedforward(self, individual, input=None):
        input_array = []

        if input is None:
            input_array = np.array(self.input, ndmin=2)
        else:
            input_array = np.array(input, ndmin=2)
        
        #xxx = np.dot(input_array, self.w1)
        
        #print('xxx', xxx)
        
        w1 = individual[0]
        w2 = individual[1]
        w3 = individual[2]
            
        layer1 = self.sigmoid(np.dot(input_array, w1) + self.bias)
        layer2 = self.sigmoid(np.dot(layer1, w2) + self.bias)
        output = self.sigmoid(np.dot(layer2, w3) + self.bias)
        
        return output;
        

        
    def calDistance(self):
        
        #print('calcdistance')
        
        kill_line = pygame.Rect(0, 0, 3, 300)
        
        #print('position_square')
        #print(square.centerx, square.centery)
        
        collided = False
        
        #squares = self.population
        
        for i in range(0, len(self.population)):
            square = []
            
            square.append(self.population[i]) #weights
            square.append(pygame.Rect(50, 250, SQUARE_WIDTH, SQUARE_HEIGHT)) #rect
            square.append(True) #alive
            square.append((random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))) #color
            square.append(0) #distance
            
            self.listSquares.append(square)

        
        while self.hasSquaresAlive == True:
            
            
            screen.fill(BLACK)
            dt = clock.tick(60)
            
            if kill_line.centerx == 600:
                kill_line.centerx = 0

            kill_line.move_ip(dt * 0.08, 0)
            pygame.draw.rect(screen, RED, kill_line)
            
            for wall in walls:
                pygame.draw.rect(screen, WHITE, wall)


            for border in borders:
                pygame.draw.rect(screen, RED, border)

            pygame.draw.rect(screen, GREEN, exit)
            
            
            for index in range(0, len(self.listSquares)):
                
                
                
                
                #print('index', index)
                
                if self.listSquares[index][2] == False:
                    continue
                
                
                #print('square')
                #print(square.centerx, square.centery)

                distance_up = distance_down = distance_right = distance_left = 0
                
                
                for wall in walls:
                    x1, x2 = self.listSquares[index][1].centerx, self.listSquares[index][1].centery
                    y1, y2 = wall.centerx, wall.centery

                    if wall.centerx == self.listSquares[index][1].centerx:
                        if self.listSquares[index][1].centery < wall.centery:
                            distance_up = math.hypot(x1-x2, y1-y2)

                        if self.listSquares[index][1].centery > wall.centery:
                            distance_down = math.hypot(x1-x2, y1-y2)

                    if wall.centery == self.listSquares[index][1].centery:
                        if self.listSquares[index][1].centerx < wall.centerx:
                            distance_right = math.hypot(x1-x2, y1-y2)

                        if self.listSquares[index][1].centerx > wall.centerx:
                            distance_left = math.hypot(x1-x2, y1-y2)

                input_array = distance_right, distance_down, distance_left, distance_up

                output = self.feedforward(self.listSquares[index][0], input_array)
                
                #print('output', output)

                action = np.argmax(output)

                #print('action', action)

                #0 direita
                #1 baixo
                #2 esquerda
                #3 cima

                pos_x = pos_y = 0

                if (action == 0):
                    pos_x = velocity * dt


                if (action == 1):
                    pos_y = (velocity * dt)

                if (action == 2):
                    pos_x = -(velocity * dt)

                if (action == 3):
                    pos_y = -(velocity * dt)

                self.listSquares[index][1].move_ip(pos_x, pos_y)    
                pygame.draw.rect(screen, self.listSquares[index][3], self.listSquares[index][1])

                
                if self.listSquares[index][1].colliderect(kill_line):
                    self.listSquares[index][2] = False


                for wall in walls:
                    if self.listSquares[index][1].colliderect(wall):
                        #print('bateu')
                        self.listSquares[index][2] = False


                for border in borders:
                    if self.listSquares[index][1].colliderect(border):
                        #print('bateu')
                        self.listSquares[index][2] = False


                #init_x, init_y = WIDTH // 2, HEIGHT // 2
                init_x, init_y = exit.centerx, exit.centery
                current_x, current_y = self.listSquares[index][1].centerx, self.listSquares[index][1].centery

                xDis = abs(current_x - init_x)
                #calcula a distancia dos eixos y
                
                yDis = abs(current_y - init_y)
                #eleva as diferenças ao quadrado, soma e calcula a raiz
                
                distance = np.sqrt((xDis ** 2) + (yDis ** 2))
                
                if distance < self.bestDistance:
                    self.bestDistance = distance
                
                self.listSquares[index][4] = distance

                #str_mov = 'distance = ' + str(round(self.distance, 2))
                #text_mov = font.render(str_mov, True, WHITE)
                #text_rect = text_mov.get_rect()
                #text_rect.center = (WIDTH // 2, HEIGHT - 60)

                #screen.blit(text_mov, text_rect)

                #squares.append(self.population[index])
                
                pygame.display.flip()

            alives = 0
                
            for alive in self.listSquares:
                if alive[2] == True:
                    #print('morreu')
                    alives += 1

            #print('alives', alives)

            if (alives == 0):
                self.hasSquaresAlive = False
                

        return True
            
    def getPathFitness(self):
        
        self.calDistance()
        
        print('bestDistance', self.bestDistance)
        
        
        for index in range(0, len(self.listSquares)):
            distance = self.listSquares[index][4]
            
            if distance != 0:
                fitness = 1 / distance
            else:
                fitness = 0
                
            self.fitnessResults[index] = fitness
            
        print('fitnessResults')
        print(self.fitnessResults)
        
        return self.fitnessResults

import pygame
import sys

BLACK = pygame.Color(0, 0, 0)
WHITE = pygame.Color(255, 255, 255)
RED = pygame.Color(255, 0, 0)
GREEN = pygame.Color(0, 255, 0)
WIDTH, HEIGHT = 600, 300

SQUARE_WIDTH, SQUARE_HEIGHT = 5, 5
WALL_WIDTH, WALL_HEIGHT = 20, 20

velocity = 0.15

pygame.init()

screen = pygame.display.set_mode((WIDTH, HEIGHT))

clock = pygame.time.Clock()

#square = pygame.Rect(50, HEIGHT -50, SQUARE_WIDTH, SQUARE_HEIGHT)


levels = [
    "BBBBBBBBBBBBBBBBBBBBBBBWWWWWWW",
    "B                      W   E W",
    "B              WWWWWWWWW    WW",
    "B             WW           W B",
    "B            WW    WWWWWWWWW B",
    "B       WWWWWW     W         B",
    "B      WW          W         B",
    "B     WW   WW     WW         B",
    "B    WW           W          B",
    "WWWWWW        WWWWW          B",
    "W             W              B",
    "W       WWWWWWW              B",
    "W       W                    B",
    "W       W                    B",
    "WWWWWWWWBBBBBBBBBBBBBBBBBBBBBB"
]


left = 0
top = 0

walls = []
borders = []
exit = 0

for row in levels:
    for col in row:
        if col == 'W':
            wall = pygame.Rect(left, top, WALL_WIDTH, WALL_HEIGHT)
            walls.append(wall)
        if col == 'B':
            border = pygame.Rect(left, top, WALL_WIDTH, WALL_HEIGHT)
            borders.append(border)
        if col == 'E':
            exit = pygame.Rect(left, top, WALL_WIDTH, WALL_HEIGHT)
        left += WALL_WIDTH
    top += WALL_HEIGHT
    if left >= WIDTH:
        left = 0


font = pygame.font.Font('freesansbold.ttf', 12)
        
class GeneticAlgorithm:
    def __init__(self, populationSize, eliteSize, mutationRate, generations):
        #self.population = population
        self.populationSize = populationSize
        self.eliteSize = eliteSize
        self.mutationRate = mutationRate
        self.generations = generations
        self.currentPopulation = []
        self.bestFitnessMovimentations = 1
        self.bestDistance = 0

    def start(self):
        self.currentPopulation = self.initialPopulation(self.populationSize, 4, 2, 4)

        for i in range(0, self.generations):
            print('generation: ', i)
            self.currentPopulation = self.nextGeneration(self.currentPopulation, self.eliteSize, self.mutationRate)

        bestIndividualIndex = self.rankIndividuals(self.currentPopulation)[0][0]
        bestIndividual = self.currentPopulation[bestIndividualIndex]
        print(bestIndividual)
        
        
    def createPath(self, listDirections):
        return random.sample(listDirections, len(listDirections))


    def initialPopulation(self, populationSize, input_size, hidden_size, output_size):
        population = []

        #for i in range(0, populationSize):
            #population.append(self.createPath(listDirections))
            
        for i in range(0, populationSize):
            #population.append(Direction(4, 2, 4))
            individual = []
            individual.append(np.random.uniform(-1., 1., (input_size,hidden_size)))
            individual.append(np.random.uniform(-1., 1., (hidden_size,hidden_size)))
            individual.append(np.random.uniform(-1., 1., (hidden_size,output_size)))
            
            population.append(individual)
            
        #print('population', population)
            
        return population


    def rankIndividuals(self, population):
        fitnessResults = {}
        
        fitnessResults = Fitness(population).getPathFitness()
        
        #for i in range(0, len(population)):
            #print('population', population[i])
            
            #print('fitness', fitnessResults)
        
        rank = sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)
        
        #print('rank')
        #print(rank)
            
        return rank


    def selection(self, populationRanked, eliteSize):
        #print('populationRanked', populationRanked)

        selectionResults = []
        df = pd.DataFrame(np.array(populationRanked), columns=["Index", "Fitness"])
        df['cum_sum'] = df.Fitness.cumsum()
        df['cum_perc'] = 100 * df.cum_sum / df.Fitness.sum()

        for i in range(0, eliteSize):
            selectionResults.append(populationRanked[i][0])
            
        #print('selection eliteSize')
        #print(len(selectionResults))

        #nSelected = len(populationRanked) - eliteSize
        nSelected = int(len(populationRanked) * 0.3)
            
        for i in range(0, nSelected):
            pick = 100 * random.random()

            for i in range(0, len(populationRanked)):
                if pick <= df.iat[i, 3]:
                    selectionResults.append(populationRanked[i][0])
                    break
                    
        print('selectionResults')
        print(len(selectionResults))
        return selectionResults

    def matingPool(self, population, selectionResults):
        matingpool = []

        for i in range(0, len(selectionResults)):
            index = selectionResults[i]
            matingpool.append(population[index])
            
        #print('matingpool')
        #print(len(matingpool))
        #print(matingpool)
            
        return matingpool

    def breed(self, parent1, parent2):
        
        #print('parent1', parent1)
        #print('parent2', parent2)
        
        child = []
        childP1 = []
        childP2 = []
        
        
        # a reproducao vai ser diferente
        # obter cada array de pesos
        # mesclar as propriedades de cada array
        # adicionar ao filho cada array
        
        
        for i in range(len(parent1)):
    
            #print('parent1[i]', parent1[i])
            
            sep = random.randint(1, len(parent1[i]) - 1)
            
            #print('sep', sep)
            
            childP1 = parent1[i][:sep]
            childP2 = parent2[i][sep:]
                        
            #print('childP1', childP1)
            #print('childP2', childP2)
            
            weights = []
            
            for j in childP1:
                weights.append(j)
                
            for k in childP2:
                weights.append(k)
            
            child.append(weights)
            
        #print('child', child)
            
        return child

    def breedPopulation(self, matingpool, eliteSize, currentGeneration):
        children = []
        
        #length = len(matingpool) - eliteSize
        length = len(currentGeneration) - eliteSize
        
        for i in range(0, eliteSize):
            children.append(matingpool[i])
            
        #print('children eliteSize')
        #print(children)
            
        #pool = random.sample(matingpool, len(matingpool))
        print('matingpool')
        print(len(matingpool))
        pool = random.sample(matingpool, length)
        
        #print('children pool')
        #print(pool)

        for i in range(0, length):
            parent1 = pool[i]
            parent2 = pool[len(matingpool)-i-1]
            child = self.breed(parent1, parent2)
            children.append(child)

        return children


    def mutate(self, individual, mutationRate):
        for i in np.arange(3):
            #print('mutate weight', i)
            for neuron in range(len(individual[i])):
                if (random.random() < mutationRate):
                    newWeight = []
                    for j in range(len(individual[i][neuron])):
                        newWeight.append(random.uniform(-1., 1))
                        #print('neuronio', individual[i][neuron])
                        
                    print('mutacao', newWeight)

                    individual[i][neuron] = newWeight

        return individual

    def mutatePopulation(self, population, mutationRate):
        mutatedPopulation = []

        for individual in range(0, len(population)):

            mutatedIndividual = self.mutate(population[individual], mutationRate)
            mutatedPopulation.append(mutatedIndividual)

        return mutatedPopulation

    def nextGeneration(self, currentGeneration, eliteSize, mutationRate):
        populationRanked = self.rankIndividuals(currentGeneration)
        #print('rank')
        #print(len(populationRanked))
        
        selectionResults = self.selection(populationRanked, eliteSize)
        
        matingpool = self.matingPool(currentGeneration, selectionResults)
        
        children = self.breedPopulation(matingpool, eliteSize, currentGeneration)
        
        return self.mutatePopulation(children, mutationRate) 


ga = GeneticAlgorithm(populationSize=100, eliteSize=20, mutationRate=0.05, generations=500)
ga.start()    
    
#while True:

    #events = pygame.event.get()
    
    #for event in events:
        #if event.type == pygame.KEYUP:
            #if event.key == pygame.K_SPACE:
          
                #ga = GeneticAlgorithm(populationSize=50, eliteSize=10, mutationRate=0.01, generations=100)
                #ga.start()


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
generation:  0
bestDistance 449.6276237065512
fitnessResults
{0: 0.0017154778604847325, 1: 0.0019038365456213533, 2: 0.0017154778604847325, 3: 0.002224062640449886, 4: 0.0019038365456213533, 5: 0.002224062640449886, 6: 0.0019038365456213533, 7: 0.002224062640449886, 8: 0.0019038365456213533, 9: 0.0019038365456213533, 10: 0.0019038365456213533, 11: 0.0017154778604847325, 12: 0.002224062640449886, 13: 0.002224062640449886, 14: 0.002224062640449886, 15: 0.002224062640449886, 16: 0.0017154778604847325, 17: 0.0017960385361444095, 18: 0.0017960385361444095, 19: 0.002224062640449886, 20: 0.0017154778604847325, 21: 0.002224062640449886, 22: 0.002224062640449886, 23: 0.0017154778604847325, 24: 0.0017960385361444095, 25: 0.0017960385361444095, 26: 0.0019038365456213533, 27: 0.002224062640449886, 28: 0.0019038365456213533, 29: 0.002224062640449886, 30: 0.0017960385361444095, 31: 0.002224062640449886, 32: 0.002224

ValueError: Sample larger than population or is negative